# CBC Mode (Cipher Block Chaining)

CBC (Cipher Block Chaining) is a block cipher mode that overcomes the pattern leakage problem of ECB. In CBC, each plaintext block is XORed with the previous ciphertext block before encryption. This dependency between blocks hides repeating patterns in the plaintext.

---

## How CBC Works

1. **Initialization:**  
   - **IV (Initialization Vector):** A random block that seeds the process.  
     For encryption, let C₀ = IV.

2. **Encryption Process:**  
   For plaintext blocks P₁, P₂, P₃, …, the encryption is computed as:
   ```
   C₁ = E_K(P₁ ⊕ C₀)
   C₂ = E_K(P₂ ⊕ C₁)
   C₃ = E_K(P₃ ⊕ C₂)
   ...
   ```
   where E_K denotes encryption with secret key K and ⊕ is the XOR operation.

3. **Decryption Process:**  
   To decrypt, the process is reversed:
   ```
   P₁ = D_K(C₁) ⊕ C₀
   P₂ = D_K(C₂) ⊕ C₁
   P₃ = D_K(C₃) ⊕ C₂
   ...
   ```
   where D_K denotes decryption with key K.

---

## Communication Flow

```text
Alice                               Bob                                 Eve
-----                               ---                                 ---
Plaintext blocks: P1, P2, P3, ...     
   |                                  
   |-- Encrypt with key K & IV -->   Ciphertext blocks: C1, C2, C3, ...  
   |                                  
   |<-- Decrypt with same K & IV --   Plaintext blocks: P1, P2, P3, ...  
```

*Eve can observe the ciphertext, but without both the key and IV she can’t recover the plaintext.*

---

## Key Parameters

- **Key (K):**  
  The symmetric key used by a block cipher (e.g., AES, DES).

- **Block Size:**  
  Typically 128 bits for AES or 64 bits for DES.

- **IV (Initialization Vector):**  
  A random, unique block for each encryption. The IV must never be reused with the same key.

- **Padding:**  
  Plaintext must be padded (e.g., using PKCS#7) to fill the final block if it is not a multiple of the block size.

---

## Security Considerations

- **Advantages:**  
  - **Pattern Concealment:** CBC successfully hides repeated plaintext blocks.
  
- **Limitations:**  
  - **IV Randomness and key (k):** Requires a **new, random and unpredictable IV** as well as a **strong key** for each encryption instance.
  - **Malleability:** Without additional integrity protection, an attacker can flip bits in the ciphertext, which produces predictable changes in the corresponding decrypted plaintext.
  
- **Recommendations:**  
  - Combine CBC with an integrity mechanism such as HMAC (Encrypt-then-MAC) for authenticated encryption.
  - Consider modern AEAD modes like AES-GCM or ChaCha20-Poly1305, which combine confidentiality with integrity checks.

---
## Encryption

![CBC Encryption](https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/CBC_encryption.svg/601px-CBC_encryption.svg.png)

---
## Decryption
![CBC Decryption](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2a/CBC_decryption.svg/601px-CBC_decryption.svg.png)

---

## Practical Usage Examples

- **Legacy Systems:**  
  - Previously used in TLS (before TLS 1.3), older encryption libraries, and legacy storage systems.
  
- **Modern Usage:**  
  - Generally being replaced by authenticated encryption methods (AEAD) that provide improved security guarantees.

---

**Summary:**  
CBC mode improves upon ECB by chaining blocks together with an IV, preventing identical plaintext blocks from encrypting to the same ciphertext. However, proper management of the IV, correct padding, and additional integrity measures (e.g., HMAC) are essential to maintain security.

In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

# --- Key (AES-128 = 16 bytes) ---
key = get_random_bytes(16)

# --- IV (must be random & unique for each encryption) ---
iv = get_random_bytes(16)

# --- Cipher ---
cipher_enc = AES.new(key, AES.MODE_CBC, iv)

# --- Plaintext (must be padded to block size) ---
plaintext = b"CBC mode : hides repeated blocks!"
pt_padded = pad(plaintext, 16)

# --- Encrypt ---
ciphertext = cipher_enc.encrypt(pt_padded)
print("Ciphertext (hex):", ciphertext.hex())

# --- Decrypt ---
cipher_dec = AES.new(key, AES.MODE_CBC, iv)
recovered = unpad(cipher_dec.decrypt(ciphertext), 16)

print("Recovered:", recovered)


Ciphertext (hex): 7f6ac59433466087e4bba81e8ce171e7a94a5a147ebf9ff66bfecc2127c28c1ca9c33b395396d3050d9dc5a1daada03b
Recovered: b'CBC mode : hides repeated blocks!'
